<a href="https://colab.research.google.com/github/alti-tude/video-slide-matching/blob/master/tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Run This Once to Initialize the Drive

#### Right click and select save to My Drive

https://drive.google.com/open?id=1J84AcNchprI0DndeaTVFSUZWF2lMo5uu

In [4]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!mkdir '/content/gdrive/My Drive/data'
!cp -r '/content/gdrive/My Drive/Data' '/content/gdrive/My Drive/data'

# Test Generator

In [10]:
import os
root = '/content/gdrive/My Drive/data/Data/Dataset/'
testroot = '/'.join(root.split('/')[:-2]) + '/new_test/'

data = []
data_ind = []

def load_tests(data, data_ind):
    for curdir in os.listdir(root):
        frames = []

        for imgs in os.listdir(root+curdir):
            if 'ppt' not in imgs:
                frames.append(root+curdir+'/'+imgs)
                data_ind.append({'ppt': root+curdir+'/ppt.jpg', 'frame':root+curdir+'/'+imgs})

        data.append({'ppt': root+curdir+'/ppt.jpg', 'frames':frames})
    

load_tests(data, data_ind)
print(data_ind[1])


{'ppt': '/content/gdrive/My Drive/data/Data/Dataset/03_7/ppt.jpg', 'frame': '/content/gdrive/My Drive/data/Data/Dataset/03_7/0.jpg'}


In [0]:
def cp(source, dest):
    ip = open(source, 'rb')
    data = b''
    while True:
        chunk = ip.read()
        if len(chunk) == 0:
            break
        data = data + chunk
    
    ip.close()
    
    op = open(dest, 'wb')
    op.write(data)

In [0]:
def write_sol(l, testroot):
    with open(testroot+'sol.txt', 'w') as sol: 
        for i in range(len(l)):
            sol.write("%s.jpg %s.jpg\n" %(i, l[i]['ppt'].split('/')[-2]))

In [0]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))



###  Rerun this for a new test

In [0]:
import random

def get_set(data_ind):
    testset_size = random.randint(100, len(data_ind))
    testset = random.sample(data_ind, testset_size)
    return testset,testset_size


In [31]:
import os

def gen_test(testroot, testset_size, testset):
    print("test size: "+str(testset_size))
    print(testroot)
    try:
        os.mkdir(testroot)
        os.mkdir(testroot+'frames/')
        os.mkdir(testroot+'ppts/')
    except Exception as e:
        print(e)
        for i in os.listdir(testroot+'frames/'):
            os.remove(testroot+'frames/'+i)
        for i in os.listdir(testroot+'ppts/'):
            os.remove(testroot+'ppts/'+i)

    out = display(progress(0, testset_size), display_id=True)

    for i in range(len(testset)):
        frame = testset[i]['frame']
        ppt = testset[i]['ppt']
        ppt_name = ppt.split('/')[-2]

        cp(ppt, testroot+'ppts/'+ppt_name+'.jpg')
        cp(frame, testroot+'frames/'+str(i)+'.jpg')
        out.update(progress(i+1, testset_size))
#         print(frame)

    write_sol(testset, testroot)



test size: 131
/content/gdrive/My Drive/data/Data/new_test/
[Errno 17] File exists: '/content/gdrive/My Drive/data/Data/new_test/'


KeyboardInterrupt: ignored

### Check OP

In [0]:
import os

def load_sol(testroot):
    solfilename = testroot+'sol.txt'

    sol = open(solfilename, 'r')
    acans = {}
    tot = 0
    while True:
        line = sol.readline()

        if line == '':
            break
        line = line.split(' ')
        
        tot += 1
        acans[line[0]] = line[1]  
    
    print("loaded: "+str(tot))
    return acans


In [0]:
def compare_sol(testroot, acans):
#     opfilename = input()
    opfilename = 'op.txt'
    opfilename = testroot+opfilename
    op = open(opfilename, 'r')
    ac = 0
    tot = 0
    errors = []
    while True:
        line = op.readline()

        if line == '':
            break

        line = line.split(' ')

        if len(line) !=2 :
            print('wrong format')
            break

        if acans[line[0]] == line[1]:
            ac+=1
        else:
            a = [line[0], line[1].strip('\n'), acans[line[0]].strip('\n')]
            errors.append(a)
            print(a)
        tot+=1

    print(ac)
    print(ac/tot*100)
    return errors, ac, tot

In [75]:
import json
import os

dump_route = root+'../test_dump.txt'

c = 0
while True:
    if os.path.isfile(root+'../breakfile'):
        break
        
    testset, testset_size = get_set(data_ind)
    gen_test(testroot, testset_size, testset)
    
    !python3 '{root}/../../../dsaa-project/main.py'
    
    acans = load_sol(testroot)
    errors, ac, tot = compare_sol(testroot, acans)
    
    if ac!=tot:
        c+=1
        with open(dump_route, 'a') as dump:
            dumpstr = json.dumps({"errors": errors, "ac":ac, "tot":tot})
            for j in range(len(errors)):
                !mkdir -p '{root}../errors/test_{c}/err_{j}'
                !cp '{testroot}frames/{errors[j][0]}' '{root}../errors/test_{c}/err_{j}/frame.jpg'
                !cp '{testroot}frames/{errors[j][1]}' '{root}../errors/test_{c}/err_{j}/incorrect_match.jpg'
                !cp '{testroot}frames/{errors[j][2]}' '{root}../errors/test_{c}/err_{j}/correct_match.jpg'
            dump.write(dumpstr + '\n')
        
    

test size: 278
/content/gdrive/My Drive/data/Data/new_test/
[Errno 17] File exists: '/content/gdrive/My Drive/data/Data/new_test/'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
1/278
2/278
3/278
4/278
5/278
6/278
7/278
8/278
9/278
10/278
11/278
12/278
13/278
14/278
15/278
16/278
17/278
18/278
19/278
20/278
21/278
22/278
23/278
24/278
25/278
26/278
27/278
28/278
29/278
30/278
31/278
32/278
33/278
34/278
35/278
36/278
37/278
38/278
39/278
40/278
41/278
42/278
43/278
44/278
45/278
46/278
47/278
48/278
49/278
50/278
51/278
52/278
53/278
54/278
55/278
56/278
57/278
58/278
59/278
60/278
61/278
62/278
63/278
64/278
65/278
66/278
67/278
68/278
69/278
70/278
71/278
72/278
73/278
74/278
75/278
76/278
77/278
78/278
79/278
80/278
81/278
82/278
83/278
84/278
85/278
86/278
87/278
88/278
89/278
90/278
91/278
92/278
93/278
94/278
95/278
96/278
97/278
98/278
99/278
100/278
101/278
102/278
103/278
104/278
105/278
106/278
107/278
108/278
109/278
110/278
111/278
112/278
113/278
114/278
115/278
116/278
117/278
118/278
119/278
120/278
121/278
122/278
1

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
1/240
2/240
3/240
4/240
5/240
6/240
7/240
8/240
9/240
10/240
11/240
12/240
13/240
14/240
15/240
16/240
17/240
18/240
19/240
20/240
21/240
22/240
23/240
24/240
25/240
26/240
27/240
28/240
29/240
30/240
31/240
32/240
33/240
34/240
35/240
36/240
37/240
38/240
39/240
40/240
41/240
42/240
43/240
44/240
45/240
46/240
47/240
48/240
49/240
50/240
51/240
52/240
53/240
54/240
55/240
56/240
57/240
58/240
59/240
60/240
61/240
62/240
63/240
64/240
65/240
66/240
67/240
68/240
69/240
70/240
71/240
72/240
73/240
74/240
75/240
76/240
77/240
78/240
79/240
80/240
81/240
82/240
83/240
84/240
85/240
86/240
87/240
88/240
89/240
90/240
91/240
92/240
93/240
94/240
95/240
96/240
97/240
98/240
99/240
100/240
101/240
102/240
103/240
104/240
105/240
106/240
107/240
108/240
109/240
110/240
111/240
112/240
113/240
114/240
115/240
116/240
117/240
118/240
119/240
120/240
121/240
122/240
1

KeyboardInterrupt: ignored

In [55]:
!python3 '{root}/../../../dsaa-project/main.py'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
caching frame
caching ppt
caching ppt
caching ppt
caching ppt
caching ppt
caching ppt
caching ppt
caching frame
caching frame
caching frame
caching frame
caching frame
caching frame


In [39]:
acans = load_sol(testroot)
compare_sol(testroot, acans)

loaded: 19
op3.txt
19
100.0


In [54]:
testset, testset_size = get_set(data_ind)
gen_test(testroot, testset_size, testset)

test size: 7
/content/gdrive/My Drive/data/Data/new_test/
[Errno 17] File exists: '/content/gdrive/My Drive/data/Data/new_test/'


In [0]:
!rm -rf '{root}../errors'